In [13]:
from datasets import load_dataset

ds = load_dataset("deepmind/aqua_rat", "raw")

ds

Generating validation split: 100%|██████████| 254/254 [00:00<00:00, 268553.87 examples/s]


DatasetDict({
    train: Dataset({
        features: ['question', 'options', 'rationale', 'correct'],
        num_rows: 97467
    })
    test: Dataset({
        features: ['question', 'options', 'rationale', 'correct'],
        num_rows: 254
    })
    validation: Dataset({
        features: ['question', 'options', 'rationale', 'correct'],
        num_rows: 254
    })
})

In [14]:
ds_train = ds['train']

In [15]:
ds_train[0]

{'question': "Two friends plan to walk along a 43-km trail, starting at opposite ends of the trail at the same time. If Friend P's rate is 15% faster than Friend Q's, how many kilometers will Friend P have walked when they pass each other?",
 'options': ['A)21', 'B)21.5', 'C)22', 'D)22.5', 'E)23'],
 'rationale': 'If Q complete x kilometers, then P completes 1.15x kilometers.\nx + 1.15x = 43\n2.15x=43\nx = 43/2.15 = 20\nThen P will have have walked 1.15*20=23 km.\nThe answer is E.',
 'correct': 'E'}

In [16]:
# Number of examples to use for the prompt set
NUM_PROMPT_EXAMPLES = 10

prompt_examples = []

for i in range(NUM_PROMPT_EXAMPLES):
    example = ds_train[i]
    question = example['question']
    options = example['options']
    rationale = example['rationale']
    correct_option = example['correct']
    
    # Map the correct option letter to the actual answer text
    option_dict = {opt[0]: opt[2:].strip() for opt in options}  # {'A': '21', 'B': '21.5', ...}
    correct_answer = option_dict[correct_option]
    
    prompt_examples.append({
        'question': question,
        'options': options,
        'rationale': rationale,
        'answer': correct_answer
    })

In [17]:
prompt_examples

[{'question': "Two friends plan to walk along a 43-km trail, starting at opposite ends of the trail at the same time. If Friend P's rate is 15% faster than Friend Q's, how many kilometers will Friend P have walked when they pass each other?",
  'options': ['A)21', 'B)21.5', 'C)22', 'D)22.5', 'E)23'],
  'rationale': 'If Q complete x kilometers, then P completes 1.15x kilometers.\nx + 1.15x = 43\n2.15x=43\nx = 43/2.15 = 20\nThen P will have have walked 1.15*20=23 km.\nThe answer is E.',
  'answer': '23'},
 {'question': 'In the coordinate plane, points (x, 1) and (5, y) are on line k. If line k passes through the origin and has slope 1/5, then what are the values of x and y respectively?',
  'options': ['A)4 and 1', 'B)1 and 5', 'C)5 and 1', 'D)3 and 5', 'E)5 and 3'],
  'rationale': 'Line k passes through the origin and has slope 1/5 means that its equation is y=1/5*x.\nThus: (x, 1)=(5, 1) and (5, y) = (5,1) -->x=5 and y=1\nAnswer: C',
  'answer': '5 and 1'},
 {'question': 'For all number

In [18]:
prompt_set = ""

for example in prompt_examples:
    question = example['question']
    options = '\n'.join(example['options'])
    rationale = example['rationale']
    answer = example['answer']
    
    prompt_set += f"Question: {question}\nOptions:\n{options}\nAnswer Explanation: {rationale}\nAnswer: {answer}\n\n"

In [21]:
# Exclude prompt examples from the main dataset
start_index = NUM_PROMPT_EXAMPLES
dataset_D = ds_train[start_index:]